In [100]:
# Librairies
import re
import pandas as pd
import ast 
import html


import os
import json
from tqdm import tqdm

# Path
path = "./fiscal_data/"
new_path = "./fiscal_data_process_20232024/"


### Fonctions génériques

In [63]:
def clean_html_column(df, column):
    """
    Nettoie les balises HTML des éléments d'une colonne d'un DataFrame.
    Args:
        df (pd.DataFrame): Le DataFrame à traiter.
        column (str): Le nom de la colonne à nettoyer.
    Returns:
        pd.Series: La colonne nettoyée (balises HTML supprimées).
    """
    def remove_html(text):
        if isinstance(text, str):
            # Supprime les balises HTML
            return re.sub(r'<.*?>', '', text)
        return text
    return df[column].apply(remove_html)


In [101]:

def clean_html_entities(df: pd.DataFrame, columns: list = None) -> pd.DataFrame:
    """
    Nettoie les entités HTML numériques et textuelles dans les colonnes spécifiées d'un DataFrame.
    - Décode les entités via html.unescape (ex: &#13; -> \\r, &amp; -> &).
    - Supprime les retours chariot (\r) et espaces multiples.

    Args:
        df (pd.DataFrame): DataFrame d'entrée.
        columns (list, optional): Liste des colonnes à nettoyer. Par défaut, toutes les colonnes.

    Returns:
        pd.DataFrame: DataFrame nettoyé.
    """
    df_clean = df.copy()
    # Si pas de colonnes spécifiées, on cible toutes les colonnes de type object
    if columns is None:
        columns = df_clean.select_dtypes(include=['object']).columns.tolist()
    
    for col in columns:
        # Convertir en str, décoder puis nettoyer les retours chariot et espaces multiples
        df_clean[col] = (
            df_clean[col]
            .astype(str)
            .apply(lambda x: html.unescape(x))  # décodage des entités
            .str.replace('\r', ' ', regex=False)  # suppression des retours chariot
            .str.replace('\n', ' ', regex=False)  # suppression des sauts de ligne
            .str.replace(r'\s+', ' ', regex=True)  # unification des espaces
            .str.strip()
        )
    return df_clean

## I) BOFIP

In [110]:
df_bofip = pd.read_csv(path + "bofip.csv")

In [111]:
df_bofip.head(1)

,division,contenu,contenu_html,debut_de_validite,serie,permalien,identifiant_juridique,type,titre
0,RFPI,L'exemple exprimé dans ce document est retiré ...,"<p class=""""paragraphe-western"""" id=""""Les_exemp...",2015-03-12,RFPI,https://bofip.impots.gouv.fr/bofip/2140-PGP.ht...,BOI-ANNX-000414,Contenu,ANNEXE - RFPI - Exemple de calcul de la déduct...


In [112]:
# Filtre sur 2023-2024

df_bofip["debut_de_validite"] = pd.to_datetime(df_bofip.debut_de_validite, format="%Y-%m-%d")
df_bofip = df_bofip[df_bofip.debut_de_validite.dt.year.isin((2023,2024))]

In [113]:
df_bofip = clean_html_entities(df_bofip,['contenu'])

In [114]:
df_bofip.to_csv(new_path+'Bofip_2023_2024.csv',index=False,sep=";")

## II) CGI

In [54]:
cgi_2023 = pd.read_csv(path + 'cgi_2023.csv')
cgi_2024 = pd.read_csv(path + 'cgi_2024.csv')

In [55]:

def process_cgi_articles(cgi_df):
    """
    Process CGI articles from a dataframe containing sections.
    
    Args:
        cgi_df (pd.DataFrame): DataFrame containing CGI sections
        
    Returns:
        pd.DataFrame: DataFrame containing all extracted articles
    """
    def extract_articles(sections_list, parent_title=None):
        if not sections_list:
            return []
        
        result = []
        for section in sections_list:
            # Si la section contient des articles, les ajouter
            if 'articles' in section and section['articles']:
                for article in section['articles']:
                    article_entry = {
                        'id': article['id'],
                        'num': article['num'],
                        'etat': article['etat'],
                        'content': article['content'],
                        'pathTitle': article['pathTitle'],
                        'executionTime': article.get('executionTime'),
                        'dereferenced': article.get('dereferenced'),
                        'cid': article.get('cid'),
                        'intOrdre': article.get('intOrdre'),
                        'section_title': section['title'],
                        'parent_title': parent_title
                    }
                    result.append(article_entry)
            
            # Récursivement chercher dans les sous-sections
            if 'sections' in section and section['sections']:
                subsections = extract_articles(section['sections'], section['title'])
                result.extend(subsections)
        
        return result

    # Appliquer la fonction à chaque ligne du dataframe
    all_articles = []
    for sections in cgi_df['sections']:
        sections_list = ast.literal_eval(sections) if isinstance(sections, str) else sections
        articles_data = extract_articles(sections_list)
        all_articles.extend(articles_data)

    # Créer un nouveau dataframe avec tous les articles
    return pd.DataFrame(all_articles)

df_cgi2023 = process_cgi_articles(cgi_2023)
df_cgi2024 = process_cgi_articles(cgi_2024)



In [57]:
df_cgi2023_2024 = pd.concat([df_cgi2023,df_cgi2024])

In [ ]:
df_cgi2023_2024['content'] = clean_html_column(df_cgi2023_2024, 'content')

In [ ]:
df_cgi2023_2024.to_csv(new_path+'CGI_2023_2024.csv',index=False,sep=";")

## III) Jurisprudence

In [71]:
df_juri = pd.read_csv('./fiscal_data/jurisprudence_global.csv')

/var/folders/p3/_m5cx42n4bzg7rgn2xc3p0s80000gn/T/ipykernel_76052/2971875856.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_juri = pd.read_csv('./fiscal_data/jurisprudence_global.csv')


In [72]:
df_juri.shape

(485952, 15)

In [73]:
# Motifs à chercher
mots_cles = [
    "code général des impôts",
    r"\bcgi\b",        # \b pour mot entier
    r"\blpf\b",
    "livre des procédures fiscales"
]
# On "échappe" proprement et on joint par |
motif = re.compile(
    "|".join(re.escape(m) for m in mots_cles),
    flags=re.IGNORECASE
)

# Filtrage vectorisé avec un seul appel
df_juri_filter = df_juri[
    df_juri["texte_integral"].str.contains(motif, na=False)
]

In [74]:
df_juri_filter.shape

(25758, 15)

In [76]:
df_juri_filter.to_csv(new_path+'Juris_filter_2023_2024.csv',index=False,sep=";")

## IV) Livre des procédures fiscales (LPF)

In [77]:
lpf2023 = pd.read_csv(path + 'lpf_2023.csv')
lpf2024 = pd.read_csv(path + 'lpf_2024.csv')

In [78]:
df_lpf2023 = process_cgi_articles(lpf2023)
df_lpf2024 = process_cgi_articles(lpf2024)

In [79]:
df_lpf = pd.concat([df_lpf2023,df_lpf2024])

In [82]:
df_lpf['content'] = clean_html_column(df_lpf, 'content')

In [84]:
df_lpf.to_csv(new_path+'LPF_2023_2024.csv',index=False,sep=";")

## V) Question_reponse AN

La donnée se décompose en Question puis en réponse

In [86]:
# Reponse
a_an_2023 = pd.read_csv('./fiscal_data/questions_reponses_AN_2023.csv')
a_an_2024 = pd.read_csv('./fiscal_data/questions_reponses_AN_2024.csv')

In [87]:
a_an = pd.concat([a_an_2023,a_an_2024])
a_an.head()

,numero_jo,date_publication,question_numero,question_page,reponse_page,reponse_texte
0,20230001,03/01/2023,922,50,50,"Aux termes de l'article 832 du code civil, l'a..."
1,20230001,03/01/2023,988,50,51,L'agriculture est l'un des secteurs particuliè...
2,20230001,03/01/2023,994,51,51,La loi n° 99-5 du 6 janvier 1999 modifiée rela...
3,20230001,03/01/2023,1497,52,52,La lutte contre les salmonelles dans les éleva...
4,20230001,03/01/2023,1691,53,54,La loi n° 99-5 du 6 janvier 1999 modifiée rela...


In [88]:
# Question



def extract_info_from_json(data):
    q = data.get("question", {})

    return {
        "uid": q.get("uid"),
        "numero": q.get("identifiant", {}).get("numero"),
        "legislature": q.get("identifiant", {}).get("legislature"),
        "type": q.get("type"),
        "rubrique": q.get("indexationAN", {}).get("rubrique"),
        "analyse": q.get("indexationAN", {}).get("analyses", {}).get("analyse"),
        "auteur_groupe": q.get("auteur", {}).get("groupe", {}).get("developpe"),
        "ministere": q.get("minInt", {}).get("developpe"),
        "date_question": q.get("textesQuestion", {}).get("texteQuestion", {}).get("infoJO", {}).get("dateJO"),
        "texte_question": q.get("textesQuestion", {}).get("texteQuestion", {}).get("texte"),
        "date_reponse": q.get("textesReponse", {}).get("texteReponse", {}).get("infoJO", {}).get("dateJO"),
        "texte_reponse": q.get("textesReponse", {}).get("texteReponse", {}).get("texte"),
        "cloture_code": q.get("cloture", {}).get("codeCloture"),
        "cloture_date": q.get("cloture", {}).get("dateCloture")
    }

def unify_jsons_to_dataframe(folder_path):
    rows = []

    for file_name in tqdm(os.listdir(folder_path)):
        if not file_name.endswith(".json"):
            continue
        file_path = os.path.join(folder_path, file_name)
        try:
            with open(file_path, "r", encoding="utf-8") as f:
                content = json.load(f)
                row = extract_info_from_json(content)
                rows.append(row)
        except Exception as e:
            continue
    df = pd.DataFrame(rows)
    return df


In [22]:

# Exemple d'utilisation
folder = "/Users/jordanchemouhoum/Downloads/Q_AN/"
df_questions = unify_jsons_to_dataframe(folder)

100%|██████████| 18710/18710 [00:03<00:00, 5063.40it/s]


In [26]:
df_questions.head(1)

,uid,numero,legislature,type,rubrique,analyse,auteur_groupe,ministere,date_question,texte_question,date_reponse,texte_reponse,cloture_code,cloture_date
0,QANR5L16QE6477,6477,16,QE,énergie et carburants,Potentiel de production d'énergies renouvelabl...,La France insoumise - Nouvelle Union Populaire...,Ministère auprès du ministre de la transition ...,2023-03-21,Mme Clémence Guetté attire l'attention de M. l...,2023-08-15,L'article 113 du projet de loi relatif à l'acc...,REP_PUB,2023-08-15


In [37]:
# Fusion des sources
df_questions['numero'] = df_questions.numero.astype(int)
a_an['question_numero'] = a_an.question_numero.astype(int)
a_an.drop_duplicates(subset="question_numero",inplace=True)

df_AN = df_questions.merge(a_an,
                           how = 'left',
                           left_on = 'numero',
                           right_on="question_numero",
                           indicator = "Merge",
                            validate = "1:1")

In [38]:
df_AN.Merge.value_counts()

Merge
both          10526
left_only      1174
right_only        0
Name: count, dtype: int64

In [39]:
df_AN = df_AN[df_AN.Merge == 'both']

In [99]:
df_AN.to_csv(new_path+'QA_AN_2023_2024.csv',sep=";",index=False)